In [10]:
# Load libraries
import casadi as ca
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [11]:
# EXP data
df_exp = pd.read_csv('Time varient simulation_v2.csv')
df_exp

,Time,PW,Fv,H2,CH4,C2H6,C2H4,C2H2,C3H8,C3H6,C4H10,C5H12,C
0,120,37.0,30,9.804,84.304,2.551,0.174,0.142,0.768,0.043,0.124,0.131,1.960
1,200,37.0,50,6.437,89.628,1.836,0.149,0.119,0.525,0.031,0.078,0.087,1.108
2,320,37.0,70,4.774,92.291,1.399,0.133,0.101,0.382,0.025,0.053,0.062,0.781
3,400,37.0,90,3.774,93.898,1.122,0.121,0.088,0.295,0.020,0.039,0.047,0.597
4,480,37.0,30,9.126,85.328,2.520,0.173,0.133,0.754,0.042,0.121,0.128,1.676
5,640,37.0,50,5.923,90.434,1.719,0.142,0.104,0.476,0.028,0.070,0.077,1.026
6,800,37.0,70,4.369,92.933,1.296,0.124,0.089,0.342,0.022,0.047,0.055,0.723
7,960,37.0,90,3.344,94.588,1.001,0.111,0.078,0.255,0.018,0.033,0.040,0.532
8,1080,37.0,30,7.898,87.296,2.168,0.159,0.109,0.633,0.035,0.096,0.107,1.498
9,1200,37.0,50,5.082,91.798,1.465,0.133,0.095,0.401,0.025,0.056,0.065,0.881


In [14]:
# Fv: mL/min, 25C(=298.15K), 1 atm(=1.01325 bar)
R = 0.08314472  # bar·L/(mol·K)
T = 25  # Celsius
T_K = T + 273.15
P = 1.01325  # bar

# Fv(mL/min) -> L/min
Fv_Lpm = df_exp['Fv'] / 1000
# n = PV/RT, V = Fv_Lpm (L/min), result: mmol/min
df_exp['F0_CH4'] = (P * Fv_Lpm) / (R * T_K) * 1e3

In [15]:
df_exp

,Time,PW,Fv,H2,CH4,C2H6,C2H4,C2H2,C3H8,C3H6,C4H10,C5H12,C,F0_CH4
0,120,37.0,30,9.804,84.304,2.551,0.174,0.142,0.768,0.043,0.124,0.131,1.960,1.22622
1,200,37.0,50,6.437,89.628,1.836,0.149,0.119,0.525,0.031,0.078,0.087,1.108,2.04370
2,320,37.0,70,4.774,92.291,1.399,0.133,0.101,0.382,0.025,0.053,0.062,0.781,2.86118
3,400,37.0,90,3.774,93.898,1.122,0.121,0.088,0.295,0.020,0.039,0.047,0.597,3.67866
4,480,37.0,30,9.126,85.328,2.520,0.173,0.133,0.754,0.042,0.121,0.128,1.676,1.22622
5,640,37.0,50,5.923,90.434,1.719,0.142,0.104,0.476,0.028,0.070,0.077,1.026,2.04370
6,800,37.0,70,4.369,92.933,1.296,0.124,0.089,0.342,0.022,0.047,0.055,0.723,2.86118
7,960,37.0,90,3.344,94.588,1.001,0.111,0.078,0.255,0.018,0.033,0.040,0.532,3.67866
8,1080,37.0,30,7.898,87.296,2.168,0.159,0.109,0.633,0.035,0.096,0.107,1.498,1.22622
9,1200,37.0,50,5.082,91.798,1.465,0.133,0.095,0.401,0.025,0.056,0.065,0.881,2.04370


In [109]:
# Components
components = ['H2', 'CH4', 'C2H6', 'C2H4', 'C2H2', 'C3H8', 'C3H6', 'C4H10', 'C5H12', 'C']

In [110]:
def acq_func(k):
    # Reactor configuration
    L = 0.15 # m, length of the DBD plasma region
    D_rod = 0.003 # m, diameter of the rod
    D_in = 0.009 # m, inner diameter of the reactor
    A = np.pi*(D_in**2 - D_rod**2)/4 # m2, cross-sectional area of the reactor
    V = A*L # m3, volume of the DBD plasma region

    # Domain
    N = 31
    dx = L / (N - 1)

    # Ideal gas law
    def pDens_Mol_Vap(T, P):
        """
        Calculate the molar density of the gas phase
        """
        R = 0.08314472  # bar·m³/(kmol·K), gas constant

        return P/(R*(T+273.15)) # kmol/m3, molar density of the gas phase

    # z_out을 매 순환마다 구하고, 이를 z_out_df에 순차적으로 저장
    z_out_list = []  # outlet mole fractions 기록 리스트
    convg_list = []
    Acc_C_list = []
    Acc_C = 0

    for i in range(len(df_exp)):
        # Operating conditions
        T = 25 # C, operating temperature
        P = 1.01325 # bar, operating pressure
        PW = df_exp['PW'][i]/(V*1e6) # W/mL, power density

        # Feed conditions
        Fv = df_exp['Fv'][i] # mL/min, feed volume flow rate
        Fv = Fv*1e-6/60 # m3/s, feed volume flow rate
        rhoF = pDens_Mol_Vap(T, P) # kmol/m3, molar density of the feed
        zF = {comp: 1 - 1e-6*9 if comp == 'CH4' else 1e-6 for comp in components}
        CF = {comp: zF[comp]*rhoF for comp in components}
        VF = Fv/A # m/s, feed superficial velocity

        # Thermodynamics
        C = {comp: ca.MX.sym(f'C_{comp}', N) for comp in components}

        rho = sum(C[comp] for comp in components)
        z = {comp: C[comp] / rho for comp in components}
        pP = {comp: P * z[comp] for comp in components}

        # Reaction
        # Reaction rate constants [kmol/s/bar^(order)/m3]
        k1f  = k[0] * 1e10    # 2CH4 -> C2H6 + H2
        k2f  = k[1] * 1e5     # C2H6 -> C2H4 + H2
        k3f  = k[2] * 1e10    # C2H4 + H2 -> C2H6
        k4f  = k[3] * 1e10    # 2CH4 -> C2H4 + 2H2
        k5f  = k[4] * 1e15    # C2H4 + 2H2 -> 2CH4
        k6f  = k[5] * 1e5      # C2H4 -> C2H2 + H2
        k7f  = k[6] * 1e10    # CH4 + C2H4 -> C3H8
        k8f  = k[7] * 1e5     # C3H8 -> C3H6
        k9f  = k[8] * 1e5     # C3H6 -> C2H4 + CH4
        k10f = k[9] * 1e15     # 2C2H4 + H2 -> C4H10
        k11f = k[10] * 1e10    # C3H8 + CH4 -> C4H10 + H2
        k12f = k[11] * 1e10    # C4H10 + CH4 -> C5H12 + H2
        k13f = k[12] * 1e5     # CH4 -> C + 2H2

        # Power exponents of reaction rates
        a1  = k[13]
        a2  = k[14]
        a3  = k[15]
        a4  = k[16]
        a5  = k[17]
        a6  = k[18]
        a7  = k[19]
        a8  = k[20]
        a9  = k[21]
        a10 = k[22]
        a11 = k[23]
        a12 = k[24]
        a13 = k[25]

        # deactivation coefficients
        d1 = 1- k[26]*Acc_C/1000
        d2 = 1- k[27]*Acc_C/1000    
        d3 = 1- k[28]*Acc_C/1000
        d4 = 1- k[29]*Acc_C/1000
        d5 = 1- k[30]*Acc_C/1000
        d6 = 1- k[31]*Acc_C/1000
        d7 = 1- k[32]*Acc_C/1000
        d8 = 1- k[33]*Acc_C/1000
        d9 = 1- k[34]*Acc_C/1000
        d10 = 1- k[35]*Acc_C/1000
        d11 = 1- k[36]*Acc_C/1000
        d12 = 1- k[37]*Acc_C/1000
        d13 = 1- k[38]*Acc_C/1000

        k1  = d1*k1f  * PW**a1
        k2  = d2*k2f  * PW**a2
        k3  = d3*k3f  * PW**a3
        k4  = d4*k4f  * PW**a4
        k5  = d5*k5f  * PW**a5
        k6  = d6*k6f  * PW**a6
        k7  = d7*k7f  * PW**a7
        k8  = d8*k8f  * PW**a8
        k9  = d9*k9f  * PW**a9
        k10 = d10*k10f * PW**a10
        k11 = d11*k11f * PW**a11
        k12 = d12*k12f * PW**a12
        k13 = d13*k13f * PW**a13

        r1 = k1 * pP['CH4'] * pP['CH4']
        r2 = k2 * pP['C2H6']
        r3 = k3 * pP['C2H4'] * pP['H2']
        r4 = k4 * pP['CH4'] * pP['CH4']
        r5 = k5 * pP['C2H4'] * pP['C2H4'] * pP['H2'] * pP['H2']
        r6 = k6 * pP['C2H4']
        r7 = k7 * pP['CH4'] * pP['C2H4']
        r8 = k8 * pP['C3H8']
        r9 = k9 * pP['C3H6']
        r10 = k10 * pP['C2H4'] * pP['C2H4'] * pP['H2']
        r11 = k11 * pP['C3H8'] * pP['CH4']
        r12 = k12 * pP['C4H10'] * pP['CH4']
        r13 = k13 * pP['CH4']

        # Mass balance
        Vx = ca.MX.sym('Vx', N)

        res = []
        for j in range(N):
            if j == 0:
                # Dirichlet B.C. at inlet:
                res.append(Vx[j] - VF)
                for comp in components:
                    res.append(C[comp][j] - CF[comp])
            elif j == N-1:
                # Neumann B.C. at outlet:
                res.append((Vx[j] - Vx[j-1])/dx)
                for comp in components:
                    res.append((C[comp][j] - C[comp][j-1])/dx)
            else:
                # Interior nodes:
                res.append(-rho[j]*((Vx[j] - Vx[j-1])/dx) + (r2[j] - r3[j] + r4[j] - r5[j] + r6[j] - r7[j] + r8[j] + r9[j] - 2*r10[j] + 2*r13[j]))
                res.append(-Vx[j]*((C['H2'][j] - C['H2'][j-1])/dx) - C['H2'][j]*((Vx[j] - Vx[j-1])/dx) + (r1[j] + r2[j] - r3[j] + 2*r4[j] - 2*r5[j] + r6[j] + r8[j] - r10[j] + r11[j] + r12[j] + 2*r13[j]))
                res.append(-Vx[j]*((C['CH4'][j] - C['CH4'][j-1])/dx) - C['CH4'][j]*((Vx[j] - Vx[j-1])/dx) + (-2*r1[j] - 2*r4[j] + 2*r5[j] - r7[j] + r9[j] - r11[j] - r12[j] - r13[j]))
                res.append(-Vx[j]*((C['C2H6'][j] - C['C2H6'][j-1])/dx) - C['C2H6'][j]*((Vx[j] - Vx[j-1])/dx) + (r1[j] - r2[j] + r3[j]))
                res.append(-Vx[j]*((C['C2H4'][j] - C['C2H4'][j-1])/dx) - C['C2H4'][j]*((Vx[j] - Vx[j-1])/dx) + (r2[j] - r3[j] + r4[j] - r5[j] - r6[j] - r7[j] - 2*r10[j]))
                res.append(-Vx[j]*((C['C2H2'][j] - C['C2H2'][j-1])/dx) - C['C2H2'][j]*((Vx[j] - Vx[j-1])/dx) + (r6[j] + r9[j]))
                res.append(-Vx[j]*((C['C3H8'][j] - C['C3H8'][j-1])/dx) - C['C3H8'][j]*((Vx[j] - Vx[j-1])/dx) + (r7[j] - r8[j] - r11[j]))
                res.append(-Vx[j]*((C['C3H6'][j] - C['C3H6'][j-1])/dx) - C['C3H6'][j]*((Vx[j] - Vx[j-1])/dx) + (r8[j] - r9[j]))
                res.append(-Vx[j]*((C['C4H10'][j] - C['C4H10'][j-1])/dx) - C['C4H10'][j]*((Vx[j] - Vx[j-1])/dx) + (r10[j] + r11[j] - r12[j]))
                res.append(-Vx[j]*((C['C5H12'][j] - C['C5H12'][j-1])/dx) - C['C5H12'][j]*((Vx[j] - Vx[j-1])/dx) + (r12[j]))
                res.append(-Vx[j]*((C['C'][j] - C['C'][j-1])/dx) - C['C'][j]*((Vx[j] - Vx[j-1])/dx) + (r13[j]))
        residuals = ca.vertcat(*res)

        # Decision variables: Vx and C
        x = ca.vertcat(Vx, *[C[comp] for comp in components])

        # Compute Jacobian: dg/dx
        constraints = ca.vertcat(*res)
        jacobian = ca.jacobian(constraints, x)

        # Objective function (minimize residual)
        objective = sum([res_i**2 for res_i in res])

        # NLP problem
        nlp = {
            'x': x,
            'f': objective,
            'g': constraints
        }

        # Solver options
        opts = {
            'ipopt.print_level': 0,  # suppress solver output
            'ipopt.max_iter': 1000,
            'print_time': 0
        }

        # Create solver
        solver = ca.nlpsol('solver', 'ipopt', nlp, opts)

        # Initial guess: Vx = VF, C = CF
        x0 = [VF] * N  # Vx initial guess
        for comp in components:
            x0.extend([CF[comp]] * N)

        # Bounds on decision variables
        # Vx: positive velocity
        lbx = [0] * N
        ubx = [ca.inf] * N
        # C: positive concentrations
        for comp in components:
            lbx.extend([0] * N)
            ubx.extend([ca.inf] * N)

        # Bounds on constraints (equality: g = 0)
        lbg = [0] * len(res)
        ubg = [0] * len(res)

        # Solve
        sol = solver(x0=x0, lbx=lbx, ubx=ubx, lbg=lbg, ubg=ubg)

        # Print solver success and objective value
        success = bool(solver.stats()['success']) if 'success' in solver.stats() else (solver.stats()['return_status'] in ['Solve_Succeeded', 'Search_Direction_Becomes_Too_Small'])

        # Extract solution vector from solver output
        x_sol = sol['x'].full().flatten()

        # Extract concentrations of each component at every position
        C_sol = {}
        idx = N
        for comp in components:
            C_sol[comp] = x_sol[idx:idx+N]
            idx += N

        # Compute total concentration at each position
        rho_sol = [sum(C_sol[comp][j] for comp in components) for j in range(N)]

        # Compute mole fractions z_sol as a dictionary of arrays (one per component)
        z_sol = {comp: [C_sol[comp][j]/rho_sol[j] if rho_sol[j] != 0 else 0.0 for j in range(N)] for comp in components}

        # Extract only outlet mole fractions (i.e., at N-1) and treat each component as a column
        z_out = {comp: z_sol[comp][-1] for comp in components}

        # Accumulateed carbon formation
        Acc_C += C_sol['C'][-1]*df_exp['Time'][i]*60

        # outlet 결과 리스트에 저장
        z_out_list.append(z_out)
        convg_list.append(success)
        Acc_C_list.append(Acc_C)

    # 모든 순환의 결과를 DataFrame으로 변환
    z_out_df = pd.DataFrame(z_out_list, columns=components)
    component_cols = z_out_df.columns
    exp_vals = df_exp[component_cols]
    mse_dict = {}
    for col in component_cols:
        mse = ((z_out_df[col]*100 - exp_vals[col]) ** 2).mean()
        mse_dict[col] = mse
    mean_mse = sum(mse_dict.values()) / len(mse_dict)

    return mean_mse, Acc_C_list, convg_list

In [111]:
def obj_func(k):
    return acq_func(k)[0]

In [ ]:
x0 = [
    6.21651338949252E-16,   # k1f (before scaling)
    1.45381304292678E-09,   # k2f (before scaling)
    1.02809258261384E-15,   # k3f (before scaling)
    6.69290771731354E-16,   # k4f (before scaling)
    2.85019436220242E-16,   # k5f (before scaling)
    9.2297699519191E-09,    # k6f (before scaling)
    6.40249201636818E-13,   # k7f (before scaling)
    8.37829309499448E-10,   # k8f (before scaling)
    1.00236478134064E-15,   # k9f (before scaling)
    3.1507112154179E-15,    # k10f (before scaling)
    1.28826018471033E-14,   # k11f (before scaling)
    3.89614664854626E-14,   # k12f (before scaling)
    2.89923516688058E-11,   # k13f (before scaling)
    1.908440317,             # a1
    1.240031702,             # a2
    1.01308532,              # a3
    1.040422716,             # a4
    0.175496697,             # a5
    0.928206422,             # a6
    0.876775871,             # a7
    1.109250254,             # a8
    0.999630013,             # a9
    0.172927302,             # a10
    2.051971156,             # a11
    2.27283981,              # a12
    1.993678731,              # a13
    0.0001,                   # d1
    0.0001,                   # d2
    0.0001,                   # d3
    0.0001,                   # d4
    0.0001,                   # d5
    0.0001,                   # d6
    0.0001,                   # d7
    0.0001,                   # d8
    0.0001,                   # d9
    0.0001,                   # d10
    0.0001,                   # d11
    0.0001,                   # d12
    0.0001,                   # d13
]
best_loss = float('inf')
best_x = x0
iter = 10
bounds = [
    (val * 0.8, val * 1.2) if val != 0 else (-0.2, 0.2)
    for val in x0
]

for i in range(iter):
    res = minimize(
        obj_func,
        x0 = x0,
        method = 'L-BFGS-B',
        bounds = bounds
    )

    print(f'trial {i+1} loss: {res.fun}, best loss: {best_loss}')
    if res.fun < best_loss:
        best_loss = res.fun
        best_x = res.x.copy()
        x0 = res.x.copy()
    else:
        x0 = best_x.copy()

    x0 = x0 * (1 + np.random.randn(len(x0)) * 0.05)

CasADi - 2025-11-03 17:03:32 WARNING("KeyboardInterruptException") [.../casadi/interfaces/ipopt/ipopt_nlp.cpp:132]


(np.float64(0.7609988190574137),
 [np.float64(5.669177953400367),
  np.float64(11.660337307765241),
  np.float64(18.676038509440637),
  np.float64(25.590982322751533),
  np.float64(48.267643963912036),
  np.float64(67.43927912731777),
  np.float64(84.97844030220357),
  np.float64(101.57420072749564),
  np.float64(152.59635423934253),
  np.float64(188.5428268165462),
  np.float64(216.6051656980848),
  np.float64(240.8070131182558),
  np.float64(312.61510549406836),
  np.float64(359.5316975450029),
  np.float64(393.00039769574647)],
 [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True])